In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
# from src.load_transform_data import get_new_ethereum_ohlc, get_new_ethereum_ohlc_2
from src.hopsworks_connections import pull_data, upload_data, pull_model, pull_model
import pandas_ta as ta
import requests
from datetime import datetime, timedelta
import time
from matplotlib import pyplot as plt


# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


In [ ]:
def get_paper_trades_df(data, start_trades_from, end_trades_on, start_at_beginning):

    start = pd.to_datetime(start_trades_from)
    end = pd.to_datetime(end_trades_on)
    predictions_df['date'] = pd.to_datetime(data['date'])
    trades_df = predictions_df[(data['date'] >= start) & (data['date'] <= end)]

    if start_at_beginning:
        trades_df = data[0:]


    results = [{
        'date': trades_df.iloc[0]['date'],
        'close': trades_df.iloc[0]['close'],
        'tmw_percent_increase': trades_df.iloc[0]['tmw_percent_increase'],
        'tmw_1_0_percent_increase_binary': trades_df.iloc[0]['tmw_1_0_percent_increase_binary'],
        'pred_tmw_1_0_percent_increase_binary': trades_df.iloc[0]['pred_tmw_1_0_percent_increase_binary'],
        'model_gains': 10000,
        'buying_every_day_gains': 10000
    }]

    for i in range(1, len(trades_df)):
        current_model_money = results[-1]['model_gains']
        current_buy_everyday_money = results[-1]['buying_every_day_gains']

        new_model_money = current_model_money * (1 + trades_df.iloc[i-1]['tmw_percent_increase']) if trades_df.iloc[i-1]['pred_tmw_1_0_percent_increase_binary'] == 1 else current_model_money
        new_buy_everyday_money = current_buy_everyday_money * (1 + trades_df.iloc[i-1]['tmw_percent_increase'])

        results.append({
            'date': trades_df.iloc[i]['date'],
            'close': trades_df.iloc[i]['close'],
            'tmw_percent_increase': trades_df.iloc[i]['tmw_percent_increase'],
            'tmw_1_0_percent_increase_binary': trades_df.iloc[i]['tmw_1_0_percent_increase_binary'],
            'pred_tmw_1_0_percent_increase_binary': trades_df.iloc[i]['pred_tmw_1_0_percent_increase_binary'],
            'model_gains': new_model_money,
            'buying_every_day_gains': new_buy_everyday_money
        })

    trading_df = pd.DataFrame(results)
    trading_df['model_gains'] = trading_df['model_gains'].map(lambda x: f'{x:,.2f}')
    trading_df['buying_every_day_gains'] = trading_df['buying_every_day_gains'].map(lambda x: f'{x:,.2f}')

    return trading_df

In [ ]:
trades_df = get_paper_trades_df(predictions_df, '04/01/2024', '04/25/2024', True)
# trading_df.to_csv('paper_trading.csv', index=False)

In [ ]:
print(precision_score(predictions_df[:-1]['tmw_1_0_percent_increase_binary'], 
                      predictions_df[:-1]['pred_tmw_1_0_percent_increase_binary']))

print(precision_score(trades_df[:-1]['tmw_1_0_percent_increase_binary'], 
                      trades_df[:-1]['pred_tmw_1_0_percent_increase_binary']))


I made a random forest classifier over the last 10 years of daily Ethereum OHLC data to predict when tomorrow's close will be 1% more than the current day's close. I created a column 'model_gains' that adds tomorrow's percent increase to the current day's close price, whenever the model predicts a 1 (buy signal). I also simulated a naive strategy of just buying the close every day.

I tested this strategy over the period 2022-04-01 to 2022-06-24, where Ethereum gradually dropped from 3451 to 1227. I'm so confused how the buying every day strategy lead to an increase of 50%.

For reference, here's a histogram of the percent close to close returns over this time period.

I can not get my head around this and am also confused how selling at the next day's close and buying right away again is any different from just holding the entire time period. Why didn't my 10k drop its value?

In [3]:
MODEL_NAME = 'random_forest_1_0_increase' # this one predicts tmw_1_0_percent_increase_binary
model = pull_model(MODEL_NAME, 1)
model

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


(RandomForestClassifier(min_samples_split=200, n_estimators=250, random_state=1),
 Model(name: 'random_forest_1_0_increase', version: 1))

In [4]:
 # model was trained without the day's data on which it was uploaded to hopsworks, so we need to subtract 1 day
timestamp = model[1].created
date_time = datetime.fromtimestamp(timestamp / 1000) - timedelta(days=1)
formatted_date = date_time.strftime('%m/%d/%Y')

In [5]:
formatted_date

'04/25/2024'

In [6]:
eth_transformed = pull_data('eth_ohlc_transformed', 1, 'eth_ohlc_transformed_view', 1)
eth_transformed

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (2.70s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,1,0,0,0,0,0,0,0,1
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,1,0,0,0,1,0,0,0,0
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3062,2024-04-22,3191.656413,3223.550024,3185.969333,3199.769590,1.210134e+10,3.780800e+06,3.844669e+11,3240.187040,0.012474,...,0,0,0,1,0,0,0,0,0,0
3063,2024-04-23,3205.104973,3261.406811,3153.784007,3218.967269,1.111125e+10,3.469997e+06,3.927751e+11,3214.336205,-0.001441,...,0,0,0,0,1,0,0,0,0,0
3064,2024-04-24,3216.267246,3289.858054,3113.541345,3138.814357,1.493166e+10,4.647113e+06,3.830686e+11,3172.757926,0.010698,...,0,0,0,0,0,1,0,0,0,0
3065,2024-04-25,3149.134891,3187.896534,3082.000384,3157.619317,1.482715e+10,4.714550e+06,3.853605e+11,3147.728366,-0.003142,...,0,0,0,0,0,0,1,0,0,0


In [9]:
eth_predictions = pull_data('eth_ohlc_predictions', 1, 'eth_ohlc_predictions', 1)
eth_predictions

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (0.73s) 


,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary
807,2015-11-15 00:00:00+00:00,0.891200,0.875000,0.921500,0.906400,0.936850,0.032503,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1226,2015-11-16 00:00:00+00:00,0.906200,0.892000,0.944700,0.929000,1.020000,0.089216,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2433,2015-11-17 00:00:00+00:00,0.924900,0.905800,1.030000,1.010000,1.000000,-0.010000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1969,2015-11-18 00:00:00+00:00,0.990000,0.940500,1.010000,0.990000,0.982750,-0.007377,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,2015-11-19 00:00:00+00:00,0.988700,0.937500,1.010000,0.955500,0.940300,-0.016165,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,2024-04-22 00:00:00+00:00,3146.652021,3134.798198,3232.067831,3198.288993,3203.014791,0.001013,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0
3063,2024-04-23 00:00:00+00:00,3200.817053,3182.763954,3223.265627,3182.763954,3214.336205,-0.001441,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
3064,2024-04-24 00:00:00+00:00,3216.267246,3113.541345,3289.858054,3138.814357,3172.757926,0.010698,0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0
3065,2024-04-25 00:00:00+00:00,3149.134891,3082.000384,3187.896534,3157.619317,3147.728366,-0.003142,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0


In [10]:
eth_predictions['avg_high_close'] = (eth_predictions['high'] + eth_predictions['close']) / 2
# Filter the DataFrame for the last two weeks
last_two_weeks_data = eth_predictions.tail(14)
last_two_weeks_data

,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary,avg_high_close
2851,2024-04-13 00:00:00+00:00,3230.000000,2883.000000,3293.000000,3013.000000,3159.500000,0.046368,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3153.000000
1977,2024-04-14 00:00:00+00:00,3011.000000,2923.000000,3166.000000,3153.000000,3185.000000,0.010047,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3159.500000
2595,2024-04-15 00:00:00+00:00,3147.000000,3035.000000,3269.000000,3101.000000,3101.500000,0.000161,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3185.000000
2572,2024-04-16 00:00:00+00:00,3098.000000,3003.000000,3121.000000,3082.000000,3051.500000,-0.009995,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3101.500000
591,2024-04-17 00:00:00+00:00,3083.000000,2926.000000,3119.000000,2984.000000,3075.500000,0.029751,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,3051.500000
2752,2024-04-18 00:00:00+00:00,2986.000000,2960.000000,3087.000000,3064.000000,3101.000000,0.011932,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3075.500000
2794,2024-04-19 00:00:00+00:00,3061.000000,2879.000000,3117.000000,3085.000000,3160.500000,0.023889,0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3101.000000
2706,2024-04-20 00:00:00+00:00,3085.000000,3025.000000,3166.000000,3155.000000,3168.500000,0.004261,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3160.500000
952,2024-04-21 00:00:00+00:00,3153.000000,3120.000000,3191.000000,3146.000000,3215.178412,0.021516,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3168.500000
644,2024-04-22 00:00:00+00:00,3146.652021,3134.798198,3232.067831,3198.288993,3203.014791,0.001013,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,3215.178412


In [11]:
eth_predictions

,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary,avg_high_close
807,2015-11-15 00:00:00+00:00,0.891200,0.875000,0.921500,0.906400,0.936850,0.032503,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.913950
1226,2015-11-16 00:00:00+00:00,0.906200,0.892000,0.944700,0.929000,1.020000,0.089216,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.936850
2433,2015-11-17 00:00:00+00:00,0.924900,0.905800,1.030000,1.010000,1.000000,-0.010000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.020000
1969,2015-11-18 00:00:00+00:00,0.990000,0.940500,1.010000,0.990000,0.982750,-0.007377,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
860,2015-11-19 00:00:00+00:00,0.988700,0.937500,1.010000,0.955500,0.940300,-0.016165,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.982750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,2024-04-22 00:00:00+00:00,3146.652021,3134.798198,3232.067831,3198.288993,3203.014791,0.001013,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,3215.178412
3063,2024-04-23 00:00:00+00:00,3200.817053,3182.763954,3223.265627,3182.763954,3214.336205,-0.001441,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,3203.014791
3064,2024-04-24 00:00:00+00:00,3216.267246,3113.541345,3289.858054,3138.814357,3172.757926,0.010698,0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,3214.336205
3065,2024-04-25 00:00:00+00:00,3149.134891,3082.000384,3187.896534,3157.619317,3147.728366,-0.003142,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,3172.757926


In [13]:
trading_df = pd.DataFrame({
    'date': [eth_predictions.iloc[0]['date']],
    'money': [10000]  # Start with $10,000
})
trading_df

,date,money
0,2015-11-15 00:00:00+00:00,10000


In [42]:
trades_df = eth_predictions[2210:]

results = [{'date': trades_df.iloc[0]['date'],
            'close': trades_df.iloc[0]['close'], 
            'tmw_percent_increase': trades_df.iloc[0]['tmw_percent_increase'], 
            'pred_tmw_1_0_percent_increase_binary': trades_df.iloc[0]['pred_tmw_1_0_percent_increase_binary'],
            'money': 10000}]

# Simulate the trading strategy
for i in range(1, len(trades_df)):
    # Get the current amount of money
    current_money = results[-1]['money']
    
    if trades_df.iloc[i-1]['pred_tmw_1_0_percent_increase_binary'] == 1:
        # Calculate new money value by adding the predicted percent increase
        new_money = current_money * (1 + trades_df.iloc[i-1]['tmw_percent_increase'])
    else:
        # No investment made, money remains the same as previous day
        new_money = current_money
    
    # Append the new date and money to the results list
    results.append({'date': trades_df.iloc[i]['date'],
                    'close': trades_df.iloc[i]['close'], 
                    'tmw_percent_increase': trades_df.iloc[i]['tmw_percent_increase'],
                    'pred_tmw_1_0_percent_increase_binary': trades_df.iloc[i]['pred_tmw_1_0_percent_increase_binary'],
                    'money': new_money})

# Convert results list to DataFrame
trading_df = pd.DataFrame(results)

In [43]:
trading_df

,date,close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,money
0,2021-12-21 00:00:00+00:00,4021.000000,0.001242,0,10000.000000
1,2021-12-22 00:00:00+00:00,3981.000000,0.035026,0,10000.000000
2,2021-12-23 00:00:00+00:00,4107.000000,-0.004525,0,10000.000000
3,2021-12-24 00:00:00+00:00,4044.000000,0.017612,0,10000.000000
4,2021-12-25 00:00:00+00:00,4101.000000,-0.004040,0,10000.000000
...,...,...,...,...,...
852,2024-04-22 00:00:00+00:00,3198.288993,0.001013,0,288663.123418
853,2024-04-23 00:00:00+00:00,3182.763954,-0.001441,0,288663.123418
854,2024-04-24 00:00:00+00:00,3138.814357,0.010698,0,288663.123418
855,2024-04-25 00:00:00+00:00,3157.619317,-0.003142,0,288663.123418


In [ ]:
eth_predictions['avg_high_close'] = eth_predictions['tmw_avg_high_close'].shift(1)
# Filter the DataFrame for the last two weeks
last_two_weeks_data = eth_predictions.tail(14)

fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(last_two_weeks_data['date'], last_two_weeks_data['close'], label='Close', marker='o', linestyle='-')
plt.plot(last_two_weeks_data['date'], last_two_weeks_data['prev_tmw_avg_high_close'], label='(High + Close) / 2', marker='o', linestyle='--')
plt.title('Ethereum Prices Over Past 2 Weeks')
plt.xlabel('Date')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
st.pyplot(fig)

In [ ]:
##### Define some additional inputs for calculating alphas
df_with_advanced_alphas_predictors = df_with_response_vars.copy()
df_with_advanced_alphas_predictors['date_key'] = df_with_advanced_alphas_predictors['date']
df_with_advanced_alphas_predictors.set_index('date', inplace=True) # necessary for ta.vwap to work
# adv{d} = average daily dollar volume for the past d days
# periods = [5, 10, 15, 20, 30, 40, 60, 81, 150, 180]]
periods = [2, 5, 10, 20, 25, 50, 100]
for period in periods:
    df_with_advanced_alphas_predictors[f'adv_{period}'] = (df_with_advanced_alphas_predictors['volume'] * df_with_advanced_alphas_predictors['close']) / ((df_with_advanced_alphas_predictors['volume'] 
                                                         * df_with_advanced_alphas_predictors['close']).rolling(window=period).mean())
    
# Calculate Daily Volume-Weighted Average Price (VWAP) - sum((high+low_close)/3) * volume / (sum of volume)
df_with_advanced_alphas_predictors['vwap'] = ta.vwap(df_with_advanced_alphas_predictors['high'], df_with_advanced_alphas_predictors['low'], 
                                                     df_with_advanced_alphas_predictors['close'], df_with_advanced_alphas_predictors['volume'])


len(df_with_advanced_alphas_predictors)

In [ ]:
##### Functions necessary for transforming input into alphas (predictors)

# today’s value of x minus the value of x d days ago
def delta(x, days):
    return x - x.shift(days)

# time-serial correlation of x and y for the past d days
def correlation(x, y, days):
    return x.rolling(window=days).corr(y)

def log(x):
    return np.log(x)

# time series rank - the rank over the past specified number of days
def ts_rank(x, days):
    return x.rolling(window=days).apply(lambda s: s.rank(ascending=False).iloc[-1])

# returns -1 if value is negative and 1 if value is positive
def sign(x):
    positives = (x > 0).astype(int)  # Converts positives to 1, else 0
    negatives = (x < 0).astype(int)  # Converts negatives to 1, else 0
    return positives - negatives  # 1 - 0 for positives, 0 - 1 for negatives, 0 - 0 for zeros

# Calculate time-series minimum over the past d days.
def ts_min(x, days):
    return x.rolling(window=days, min_periods=1).min()

# Calculate time-series maximum over the past d days
def ts_max(x, days):
    return x.rolling(window=days, min_periods=1).max()

# Calculate time-series sum over past d days
def sum(x, days):
    return x.rolling(window=days).sum()

# Calcluate standard deviation over past d days
def stddev(x, days):
    return x.rolling(window=days).std()

# value of x d days ago
def delay(x, days):
    return x.shift(days)

# Safe less than (<) comparison that preserves NaN values """
def safe_lt(series1, series2):
    return (series1 < series2) & pd.notna(series1) & pd.notna(series2)

# Safe less than or equal to (<=) comparison that preserves NaN values
def safe_le(series1, series2):
    return (series1 <= series2) & pd.notna(series1) & pd.notna(series2)

#
def safe_gt(series1, series2):
    return (series1 > series2) & pd.notna(series1) & pd.notna(series2)



In [ ]:
def add_predictor_vars2(data):
    df = data.copy()

    ##### Add columns that will be inputs to the 101 formulaic alphas from "101 Formulaic Alphas, by Zura Kakushadze, 2015"
    df.set_index('date', inplace=True) # necessary for ta.vwap to work
    periods = [20]
    for period in periods:
        df[f'adv_{period}'] = (df['volume'] * df['close']).rolling(window=period).mean()
    # Calculate Daily Volume-Weighted Average Price (VWAP) - sum((high+low_close)/3) * volume / (sum of volume)
    df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])


    ##### Functions necessary for transforming input into alphas (predictors)
    # cross-sectional rank - the rank over the whole pandas series
    def rank(x):
        return x.rank(ascending=False) # 1 is assigned to the smallest value
    # today’s value of x minus the value of x d days ago
    def delta(x, days):
        return x - x.shift(days)
    # time-serial correlation of x and y for the past d days
    def correlation(x, y, days):
        return x.rolling(window=days).corr(y)
    def log(x):
        return np.log(x)
    # time series rank - the rank over the past specified number of days
    def ts_rank(x, days):
        return x.rolling(window=days).apply(lambda s: s.rank(ascending=False).iloc[-1])
    # returns -1 if value is negative and 1 if value is positive
    def sign(x):
        positives = (x > 0).astype(int)  # Converts positives to 1, else 0
        negatives = (x < 0).astype(int)  # Converts negatives to 1, else 0
        return positives - negatives  # 1 - 0 for positives, 0 - 1 for negatives, 0 - 0 for zeros
    # Calculate time-series minimum over the past d days.
    def ts_min(x, days):
        return x.rolling(window=days, min_periods=1).min()
    # Calculate time-series maximum over the past d days
    def ts_max(x, days):
        return x.rolling(window=days, min_periods=1).max()
    # Alpha#2: (-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))
    df["alpha_2"] = -1 * correlation(rank(delta(log(df['volume']), 2)), rank(((df['close'] - df['open']) / df['open'])), 6)

    # Alpha#3: (-1 * correlation(rank(open), rank(volume), 10))
    df["alpha_3"] = -1 * correlation(rank(df['open']), rank(df['volume']), 10)

    # Alpha#4: (-1 * ts_Rank(rank(low), 9))
    df["alpha_4"] = -1 * ts_rank(rank(df['low']), 9)

    # Alpha#5: (rank((open - (sum(vwap, 10) / 10))) * (-1 * abs(rank((close - vwap)))))
    df["alpha_5"] = rank(df['open'] - (sum(df['vwap'], 10) / 10)) * (-1 * abs(rank((df['close'] - df['vwap']))))

    # Alpha#6: (-1 * correlation(open, volume, 10))
    df["alpha_6"] = -1 * correlation(df['open'], df['volume'], 10)

    # Alpha#7: ((adv20 < volume) ? ((-1 * ts_rank(abs(delta(close, 7)), 60)) * sign(delta(close, 7))) : (-1 * 1))
    df["alpha_7"] = np.where(
        df['adv_20'] < df['volume'],
        -1 * ts_rank(abs(delta(df['close'], 7)), 60) * sign(delta(df['close'], 7)),
        -1)

    # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1))))
    df["alpha_9"] = np.where(
        ts_min(delta(df['close'], 1), 5) > 0,
        delta(df['close'], 1),
        np.where(
            ts_max(delta(df['close'], 1), 5) < 0,
            delta(df['close'], 1),
            -1 * delta(df['close'], 1)
        )
    )
    return df
